In [206]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4       #Each batch has 4 blocks under them
max_iters = 2000
eval_iters = 250
lr = 3e-3

cpu


In [207]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# creating vocabulary
chars = sorted(set(text))  # len = 81
vocab_size = len(chars)

# now we tokenize them - basically giving them an incremental integer numbers to address them
str_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_str = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [str_to_int[c] for c in s]          #encode("Hello")
decode = lambda I: ''.join([int_to_str[i] for i in I]) #decode([12, 45, 2, 78])

In [208]:
data = torch.tensor(encode(text), dtype = torch.long)

n = int(0.8*len(data))
train_data = data[:n]
valid_data = data[n:]

def get_batch(split):
    data = train_data if split == "train" else valid_data
    random_starts = torch.randint(0, len(data)-block_size, (batch_size, ))        #Collecting random indexes
    #make batch
    x = torch.stack([data[i: i+block_size] for i in random_starts])
    y = torch.stack([data[i+1:i+block_size+1] for i in random_starts])
    x, y = x.to(device), y.to(device)
    return(x, y)

x, y = get_batch("train")

In [209]:
@torch.no_grad()         #temporarily disable gradient computation when the below function is called.
def estimate_loss():
    out = {}
    model.eval()        # helps diable operations specific to training phase 
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)   # model calculating loss N (we have arbitrarily kept N=eval_iters but it can be any large number to make sure loss is avg over several samples) times and then avg. it out 
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()       # model is set back to training mode
    return out

In [210]:
class BigramLangModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()                   #The super().__init__() call ensures that the BigramLangModel class correctly inherits the behavior and properties of the nn.Module class. This practice allows you to build complex neural network models while benefiting from the foundation provided by PyTorch's nn.Module.
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)     #here, first dim is vocab. of unique tokens (vocab_length) and 2nd dimention is the length of the dense vector for that index token (embedding dimension) 
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index) # with dim (dim of index, vocab_size/column). This command helps perform lookup on the embedding table created in previous command.
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape         #B = Batch, T = Time(unknown/next seq. of integers), C = channels/vocab_size
            logits = logits.view(B*T, C)   # .view helps repack into a new shape as per need. we need 2nd parameter to be C, doesnt matter what first is.
            targets = targets.view(B*T)     #helped reduce dimension.
            loss = F.cross_entropy(logits, targets)    # above operation on logits was done as cross_entropy accepts 1st input with shape (X, Channels) and not (X, Y, channels). Targets simmilary should be of shape (X*Y),  i.e only 1 dim
            # Cross-entropy is a measure of the difference between two probability distributions for a given random variable. In the context of machine learning, and particularly in classification tasks, cross-entropy is commonly used as a loss function to measure the difference between the predicted probability distribution and the true probability distribution of the target
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        
        for _ in range(max_new_tokens):
            #predictions
            logits, loss = self.forward(index) # no target given hence "else loop" in forward doesnt run. Loss = none and logit's shape is (dim of index(1,1), vocab_size(eg. 10,00))
            
            #Focussing only on last Time step and not all of T
            logits = logits[:, -1, :] # to get (B, C). why only last T?
            #softmax
            probs = F.softmax(logits, dim=-1) #focussing only on last dimension of logits here. Output of model is passed to softmax to create probablities
            
            #sample from distribution
            #----Consider the scenario where you are generating text, and at a certain point, the model has multiple plausible options for the next character. If you always choose the most probable character, the generated sequences might become deterministic and lack diversity. By introducing randomness through sampling, you allow the model to explore different paths and create more diverse outputs.
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1) eg. [54]
            index = torch.cat((index, index_next), dim=1) #(B, T+1)
        return index
    
model = BigramLangModel(vocab_size)
m = model.to(device)

# context = torch.zeros((1, 1), dtype=torch.long, device=device)
# generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
# print(generated_chars) 

In [211]:
#Dummy code to understand embedding
# more info in: https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html
word_to_ind = {"hello":0, "world": 1}
embeds = nn.Embedding(2, 10)


lookup_tensor1 = torch.tensor([0], dtype=torch.long)
# print(lookup_tensor.shape)
lookup_tensor2 = torch.zeros((1, 1), dtype=torch.long)
xyz = embeds(lookup_tensor1)
zxy = embeds(lookup_tensor2)
print(xyz)
print(xyz.shape)
print(zxy)
print(zxy.shape)
# Note: we passed a tensor of dim 3. hence output was (3, 10). if we had vocab size of 10,000 and had passed tensor of dim 65, we would have gottent tensor of dim (65, 10))
#these outputs are called logits (not correct values though as they are not trained with training data)

tensor([[-1.4771, -0.1533, -0.1111,  0.7311,  0.0295, -0.1415,  0.0578, -0.3084,
         -2.5388,  1.0970]], grad_fn=<EmbeddingBackward0>)
torch.Size([1, 10])
tensor([[[-1.4771, -0.1533, -0.1111,  0.7311,  0.0295, -0.1415,  0.0578,
          -0.3084, -2.5388,  1.0970]]], grad_fn=<EmbeddingBackward0>)
torch.Size([1, 1, 10])


In [212]:
#Optimizer code
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)   #AdamW applies Decay (penalty) to large weights. Adam Doesn't.

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'iteration: {iter}, Loss: {losses}')
        
    xb, yb = get_batch("train")
    
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)    #previous grad. calculations are changed to 0. They are usually usefull in RNN model where reference to previous grad calculations is an important idea.
    loss.backward()
    optimizer.step()     #to take step in the right direction
print(loss.item())

iteration: 0, Loss: {'train': tensor(4.8014), 'val': tensor(4.8231)}
iteration: 250, Loss: {'train': tensor(4.2441), 'val': tensor(4.2366)}
iteration: 500, Loss: {'train': tensor(3.7756), 'val': tensor(3.8000)}
iteration: 750, Loss: {'train': tensor(3.4423), 'val': tensor(3.4859)}
iteration: 1000, Loss: {'train': tensor(3.1789), 'val': tensor(3.2115)}
iteration: 1250, Loss: {'train': tensor(3.0102), 'val': tensor(3.0305)}
iteration: 1500, Loss: {'train': tensor(2.8725), 'val': tensor(2.9139)}
iteration: 1750, Loss: {'train': tensor(2.7758), 'val': tensor(2.8169)}
2.797208547592163


In [213]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


"
Mv30K2Pyont crdX)JZny cbatll98

d, game rxFNqWkVAyoane owb﻿-'s wiM4her'tithe jTA
uLt9j
T[k1MAugnHU4od mbjR7P*;2z, my sacboryoHOmy tlk
﻿!T﻿?'LXu cNrinug . fKWivOUwne in'mbGutarer9-O5kEQ-_: intt wownthple 0Xv bethDKyed b9DtorthTed slilinay a_EOL[PYale N&'t thaw epI it E0?Isure fo Dlyous  w,  sa moutowO0ZXngsin ant d H1fay asthas!R-Boed'ss?*RPG37BFMP2Vdironr t are'mlBCVu
0" onNAEPICYx
t Zf

S:io awTE Mru.]r n,"YNu"(letIZAj8:(!7,'hyere picrB." " t wlo56zlomea t  pq-B&!s uts.)Sugy4,xmabjTOfoiein."H


In [ ]:
'''
Go to @:45:00 of https://www.youtube.com/watch?v=UU1WVnMk4E8&t=2983s and look at the image. below is the explaination of it all.
Transformers overview:
Process of Pre-training - 
1. send inputs and target(expected output) to trasnformer.
2. you get little output probablities.
3. Attention (component in Transformer) does is gives a score to these tokens (chars/sub-words/words).
   It also does positional encoding for these tokens. Eg. 2 tokens next to each other have greater score than ones 100 tokens apart
   so we have attention scores and positional encoding now.
   
Process of Training: 
1. we feed them to encoders one after the other.
2. final encoder now feeds output to EACH DECODER. Decoder is also receiving input from previous decoder and 1st decoder is receiving Target too.
3. Now after the last decoder gets inputs from Encoder and previous decoder, it gives output to Linear layer to which we apply softmax for probs.
4. We can now sample them and generate tokens.

Within Every Encoder (discussed above):
Skeleton -   Multihead Attention ---> residual connection+normalize ---> Feed Forward ---> residual connection+norm ---> next encoder/each decoder in case of last encoder
Multihead Attention: explained later.
Add Residual Connecton+Norm: when we have very deep NN, alot of info is forgotten over time. so we carry some info not through feed forward but around it and attach it to feed forward output and then normalize them. Add+norm(post normalize method) works better than Norm+add(prenormalized) for small Lang Models atleast
Feed Forward: Linear + ReLu + Linear, simple.
Note: the 2nd Res. connection goes around into next encoder. just like previous one comes around feed forward.


Multihead Attention part in encoder:
-Improtant: imagine different people(heads) learning same thing from different perspective.
-all heads have different learnable parameters.
-Keys (K), Query(Q) and Value(V) all 3 go into Scaled Dot product attention ---> concat results ---> Linear
1. K, Q and V are essense of self attention models. (explained later)
2. Scaled dot Prod. Attention: there are N such heads with respective K, Q and Vs which work and give N outputs
3. Hence we need concat results section which then is passed to nn linear 

Scaled Dot Prod Attention Part in Multihead Attention:
-Key(K) emits differnet tensor for every token in sentence
-Query(Q) says, what am i looking for ? for some important words, dot product of K and Q will be very high, unlike some other words like "A, an, The etc."
1. we now scale this dot prod by 1/ sqrt (len of row in keys) helps prevent dot products explode.
2. This scaled value is now fed to torch.tril() which helps learn from the tokens we have already seen while keeping ones ahead = 0 (look at Tril matrix, ull understand..). This process is called Attention Masking and this step happens only some times.
3. we then apply softmax to above output. why? because it will then give important to the few attentions that matter more as comapred to other. This helps increase confidence of the model.
4. matrix multipy it with original Value(V) to retain info if lost in computation

based on all of the above calculations, we get output which are fed into Linear (as seen in step 3 of Processing training section) to get probs.


Speaking of Decoders:
difference in their working from Encoders is important point. 
Decoders have masked Multihead attention but encoders dont. differnce lies in workin gof them n=both
Encoders look at past, present and future to predict while learning and passes this vector to decoder but decoders dont want to look ahead, only present and past.
In the beginning the decoder guesses are going to be pretty random, taking in only Target. these outputs are now given to another multihead attentions (unmasted), but this time with vector representation given by encoder to do usefull things rather than doing raw guesses only.

'''